<a href="https://colab.research.google.com/github/Icesmoke/my-arduino-/blob/master/tele2_status_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install xlsxwriter

In [0]:
import pandas as pd
import numpy as np
import xlsxwriter as xlsw
import os

# импорт для работы с датами
import datetime
from datetime import datetime, timedelta, date

# настройка параметров выводв информации
pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 190)

import matplotlib.pyplot as plt
%matplotlib inline

# смена рабочей директории
#%cd h:\ЛЦ\analysis\tele2\Files

#os.chdir('h:\\ЛЦ\\analysis\\tele2\\Files')

def alerts(uroven, mean):
    if uroven > (mean * 1.3):
        mes = 'Внимание'
    else:
        mes = 0
    return mes

def time_alerts(dur):
    if dur > 30000:
        mes = 'сбой'
    elif dur > 600:
        mes = 'Внимание'
    return mes



# Чтение файла
report0 = pd.read_excel('/content/43 nedelya.xlsx')

report = report0.copy()
report = report.fillna(0)

In [0]:
#Разделение столбца площадки и оператора
report_apend = report['ФИО оператора '].str.split('_',expand=True)
report_apend = report_apend[[0,1]]
report_apend.columns = ['Place', 'Operator']
report = pd.concat([report, report_apend],axis=1)
report.drop(['Запись разговора', 'ФИО оператора '], axis =1, inplace = True)

# переименование столбцов
report.columns = ['call_number', 'date', 'Call_result', 'duration_waiting', 'duration_call_min', 
                  'duration_wrapup_min', 'duration_call_sec', 'hold_sec', 'duration_wrapup_sec', 'Project', 'Place', 'Operator']
report = report[['call_number', 'date', 'Project', 'Place', 'Operator', 'Call_result', 'duration_waiting', 'duration_call_min', 'duration_wrapup_min', 'duration_call_sec', 'duration_wrapup_sec']]

# групприровка операторов с подсчетом количества звонков
report_operator = report.groupby(['Operator', 'Place']).count().reset_index().fillna(0)
report_operator1 = report_operator.copy()

# Формирование отдельных списков с выделением статусов и длительностью звонка
avtootvet = report.Call_result == 'Автоответчик'
avtootvet_report = report[avtootvet]

avtootvet_dlit = avtootvet_report.duration_call_sec > 12
avtootvet_report = avtootvet_report[avtootvet_dlit]


ne_berut = report.Call_result == 'Не берут трубку'
ne_berut_report = report[ne_berut]

ne_berut_dlit = ne_berut_report.duration_call_sec > 12
ne_berut_report = ne_berut_report[ne_berut_dlit]

obryv = report.Call_result == 'Обрыв связи'
obryv_report = report[obryv]

obryv_dlit = obryv_report.duration_call_sec > 12
obryv_report = obryv_report[obryv_dlit]

avtootvet_operator = avtootvet_report.groupby(['Operator', 'Place']).count().reset_index().fillna(0)
ne_berut_operator = ne_berut_report.groupby(['Operator', 'Place']).count().reset_index().fillna(0)
obryv_operator = obryv_report.groupby(['Operator', 'Place']).count().reset_index().fillna(0)

avtootvet_operator_sel = avtootvet_operator[['Operator', 'Place', 'call_number']]
avtootvet_operator_sel.columns = ['Operator', 'Place', 'call_number_avtootvet']

ne_berut_operator_sel = ne_berut_operator[['Operator', 'Place', 'call_number']]
ne_berut_operator_sel.columns = ['Operator', 'Place', 'call_number_ne_berut']

obryv_operator_sel = obryv_operator[['Operator', 'Place', 'call_number']]
obryv_operator_sel.columns = ['Operator', 'Place', 'call_number_obryv']

report_operator_sel = report_operator1[['Operator', 'Place', 'call_number']]


#  Определение средней длительности звонков со статусом отказ от услуги
otkaz = report.Call_result == 'Отказ от предложения'
otkaz_report = report[otkaz]
otkaz_operator = otkaz_report.groupby(['Operator', 'Place']).mean().reset_index()
otkaz_operator = otkaz_operator[['Operator', 'Place', 'duration_call_sec', 'duration_wrapup_sec']]
otkaz_operator = otkaz_operator.round({'duration_call_sec': 1, 'duration_wrapup_sec': 1})
otkaz_operator.columns = ['Оператор', 'Площадка', 'Средняя продолжительность звонка Сек', 'Средняя продолжительность постобработки Сек']

# Слияние различных списков по разным статусам в одну таблицу
operator_analysis1 = pd.merge(avtootvet_operator_sel, report_operator_sel, how = 'left', on= 'Operator')
operator_analysis2 = pd.merge(operator_analysis1, ne_berut_operator_sel, how = 'left', on= 'Operator')
operator_analysis = pd.merge(operator_analysis2, obryv_operator_sel, how = 'left', on= 'Operator')

operator_analysis.drop(['Place_y'], axis =1, inplace = True)
operator_analysis.columns = ['Operator', 'Place', 'call_number_avtootvet', 'call_number', 'Place_x', 'call_number_ne_berut', 'call_number_obryv']
del operator_analysis['Place_x']
operator_analysis = operator_analysis[['Operator', 'Place', 'call_number', 'call_number_avtootvet', 'call_number_ne_berut', 'call_number_obryv']]
operator_analysis = operator_analysis.fillna(0)

#  Расчет доли количества проставленных статусов в общем количестве звонков
analysis = operator_analysis.copy()
analysis.loc[:, 'uroven_avtootvetchikov'] = analysis['call_number_avtootvet'] / analysis['call_number']
analysis.loc[:, 'uroven_ne_berut'] = analysis['call_number_ne_berut'] / analysis['call_number']
analysis.loc[:, 'uroven_obryv'] = analysis['call_number_obryv'] / analysis['call_number']

# Определяем средний уровень 
avtootvet_mean = analysis.uroven_avtootvetchikov.mean()
ne_berut_mean = analysis.uroven_ne_berut.mean()
obryv_mean = analysis.uroven_obryv.mean()

# Выставляем алерты по превышению среднего уровня на 20%
analysis.loc[:, 'avtootvetchik_alert'] = analysis.apply(lambda row: alerts(row['uroven_avtootvetchikov'], avtootvet_mean), axis = 1)
analysis.loc[:, 'ne_berut_alert'] = analysis.apply(lambda row: alerts(row['uroven_ne_berut'], ne_berut_mean), axis = 1)
analysis.loc[:, 'obryv_alert'] = analysis.apply(lambda row: alerts(row['uroven_obryv'], obryv_mean), axis = 1)

analysis = analysis.round({'uroven_avtootvetchikov': 3, 'uroven_ne_berut': 3, 'uroven_obryv': 3})

analysis.columns = ['Операторы', 'Площадка', 'Количество звонков всего', 'Количество статусов Автоответчик', 
                    'Количество статусов Не берут трубку', 'Количество статусов Обрыв связи', 'Доля Автоответчик', 
                    'Доля Не берут', 'Доля Обрыв связи', 'Часто ставит Автоответчик', 'Часто ставит Не берут трубку', 'Часто ставит Обрыв связи']

# Определение звонков со статусами автоответчик и не берут трубку с длительностью свыше 45 сек
avtootvet_super_dlit = avtootvet_report.duration_call_sec > 45
avtootvet_report_super = avtootvet_report[avtootvet_super_dlit]
avtootvet_report_super = avtootvet_report_super[['call_number', 'date', 'Project', 'Place', 'Operator', 'duration_call_sec', 'duration_wrapup_sec']]

ne_berut_super_dlit = ne_berut_report.duration_call_sec > 45
ne_berut_report_super = ne_berut_report[ne_berut_super_dlit]
ne_berut_report_super = ne_berut_report_super[['call_number', 'date', 'Project', 'Place', 'Operator', 'duration_call_sec', 'duration_wrapup_sec']]


a_champ = avtootvet_report_super.copy()
avtootvet_champ = a_champ.groupby(['Operator', 'Place']).count().reset_index()
avtootvet_champ = avtootvet_champ[['Operator', 'Place', 'call_number']]


nb_champ = ne_berut_report_super.copy()
ne_berut_champ = nb_champ.groupby(['Operator', 'Place']).count().reset_index()
ne_berut_champ = ne_berut_champ[['Operator', 'Place', 'call_number']]


ne_berut_champ.columns = ['Оператор', 'Площадка', 'Количество превышений']
avtootvet_champ.columns = ['Оператор', 'Площадка', 'Количество превышений']
avtootvet_report_super.columns = ['Номер', 'Дата время', 'Проект', 'Площадка', 'Оператор', 'Длительность звонка Сек', 'Длительность постобработки Сек']
ne_berut_report_super.columns = ['Номер', 'Дата время', 'Проект', 'Площадка', 'Оператор', 'Длительность звонка Сек', 'Длительность постобработки Сек']

In [0]:
# Запись в файл
writer = pd.ExcelWriter('Анализ статусов Теле2.xlsx', engine='xlsxwriter')
analysis.to_excel(writer, 'Анализ', index = False)
avtootvet_champ.to_excel(writer, 'Автоответчик колич превышений', index = False)
avtootvet_report_super.to_excel(writer, 'Автоответчик превыш длит', index = False)
ne_berut_champ.to_excel(writer, 'Не Берут колич превышений', index = False)
ne_berut_report_super.to_excel(writer, 'Не берут превыш длит', index = False)
otkaz_operator.to_excel(writer, 'Отказ средн длит звонка', index = False)
#s2.to_excel(writer, 'prostoy')



writer.save()